<a href="https://colab.research.google.com/github/Aerospacerr/-PS-S3E13-/blob/main/Stock_Price_Scraper_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialization and Imports

In [1]:
# ### Initialization and Imports
# Update the system
!sudo apt -y update

# Install required packages
!sudo apt install -y wget curl unzip

# Download and install libu2f-udev dependency for Chrome
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb

# Download and install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb

# Download and install ChromeDriver
!wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

# Install selenium and chromedriver_autoinstaller
!pip install selenium chromedriver_autoinstaller
!pip install yfinance

from IPython.display import clear_output
#clear_output()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,129 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 https://r2u.stat

In [2]:
from IPython.display import clear_output
import sys
import time
from urllib.parse import quote
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
import yfinance as yf
from google.colab import drive

In [3]:
# Add chromedriver to system path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


# Configure Chrome options for headless browsing
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Install ChromeDriver automatically
chromedriver_autoinstaller.install()

# Initialize Chrome driver
driver = webdriver.Chrome(options=chrome_options)

In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
from urllib.parse import quote
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
import yfinance as yf

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [5]:
#Scraping

trade_data = []

### Data Scraping

In [6]:
driver.get('https://www.quiverquant.com/congresstrading/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

politician_links = []

for politician_link in soup.find_all('a', attrs={'class': 'flex-row-center'}):
    politician_links.append('https://www.quiverquant.com/' + quote(politician_link.get('href').replace('../', '')))

print(f'Found {len(politician_links)} politicians')
print(f'First politician: {politician_links[0]}')

Found 87 politicians
First politician: https://www.quiverquant.com/congresstrading/politician/Josh%20Gottheimer-G000583


In [7]:
pbar = tqdm(politician_links)

for politician_link in pbar:
    while True:
        driver.get(politician_link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        script_tag = soup.find('script', string=re.compile('let tradeData ='))

        try:
            script_content = script_tag.string
            pbar.set_description(f'Loaded {politician_link}')
            break
        except:
            pbar.set_description(f'Failed to load {politician_link}. Retrying in 10 seconds...')
            time.sleep(10)

    trade_data_match = re.search(r'let tradeData = (\[.*?\]);', script_content, re.DOTALL)
    trade_data_json = trade_data_match.group(1)

    trade_data_by_politician = json.loads(trade_data_json)

    for td in trade_data_by_politician:
        trade_data.append(
            {
                'Politician Profile': politician_link,
                'Stock Ticker': td[0],
                'Transaction': td[1],
                'Disclosed': td[2],
                'Traded': td[3],
                'Description': td[4],
                'Politician': td[6],
                'House Information': td[7],
                'Stock': td[8],
                'Amount Range': td[10],
                'Industry': td[13],
                'Amount': td[14]
            }
        )

    time.sleep(1)

Failed to load https://www.quiverquant.com/congresstrading/politician/Suzan%20K.%20DelBene-D000617. Retrying in 10 seconds...:  13%|█▎        | 11/87 [01:15<08:39,  6.83s/it]


KeyboardInterrupt: 

### Data Exploration

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# Load the dataset
file_path = '/content/drive/MyDrive/Upwork/Stock Data Analysis/transaction_scraped.csv'
transactions = pd.read_csv(file_path)

# Display the first few rows of the dataframe
transactions.sample(50)

,Politician Profile,Stock Ticker,Transaction,Disclosed,Traded,Description,Politician,House Information,Stock,Amount Range,Industry,Amount
19129,https://www.quiverquant.com/congresstrading/po...,AAPL,Purchase,2020-05-29 00:00:00,2020-04-03,NaN,Katherine M. Clark,House-C001101-525,APPLE INC.,"$1,001 - $15,000",Information Technology,8000.0
2506,https://www.quiverquant.com/congresstrading/po...,NSC,Sale,2019-01-04 00:00:00,2018-12-10,NaN,Josh Gottheimer,House-G000583-311,NORFOLK SOUTHERN CORPORATION,"$1,001 - $15,000",Industrials,8000.0
3544,https://www.quiverquant.com/congresstrading/po...,RACE,Purchase,2022-11-10 14:56:00,2022-10-06,NaN,Tommy Tuberville,Senate-T000278-590,Ferrari N.V. Common Shares,"$1,001 - $15,000",Consumer Discretionary,8000.0
7756,https://www.quiverquant.com/congresstrading/po...,MSFT,Purchase,2022-08-10 00:00:00,2022-07-27,NaN,Kathy E. Manning,House-M001135-344,MICROSOFT CORPORATION,"$1,001 - $15,000",Information Technology,8000.0
18004,https://www.quiverquant.com/congresstrading/po...,VTIP,Sale (Full),2022-07-13 10:57:00,2022-06-30,NaN,Lindsey Graham,Senate-G000359-3,Vanguard Short-Term Inflation-Protected Securi...,"$15,001 - $50,000",NaN,32500.0
18345,https://www.quiverquant.com/congresstrading/po...,VOD,Purchase,2018-04-18 00:00:00,2018-04-09,NaN,Debbie Dingell,House-D000624-156,VODAFONE GROUP PLC - AMERICAN DEPOSITARY SHARE...,"$1,001 - $15,000",Communication Services,8000.0
46,https://www.quiverquant.com/congresstrading/po...,HQY,Sale,2024-07-09 00:00:00,2024-06-21,NaN,Josh Gottheimer,House-G000583-2750,"HEALTHEQUITY, INC. - COMMON STOCK","$1,001 - $15,000",Health Care,8000.0
11211,https://www.quiverquant.com/congresstrading/po...,NaN,Purchase,2019-04-02 09:32:00,2019-03-29,Rate/Coupon: 8.35% Matures: 0...,Thomas R. Carper,Senate-C000174-642,MS Contingent Autocall on Boeing,"$15,001 - $50,000",NaN,32500.0
5604,https://www.quiverquant.com/congresstrading/po...,PXD,Exchange,2024-06-07 00:00:00,2024-05-03,EXCHANGED FOR 2.3234 SHARES OF...,Kevin Hern,House-H001082-802,PIONEER NATURAL RESOURCES COMPANY COMMON STOCK,"$100,001 - $250,000",Energy,175000.0
12539,https://www.quiverquant.com/congresstrading/po...,ARLP,Sale,2017-04-11 00:00:00,2017-03-09,NaN,Virginia Foxx,House-F000450-180,"ALLIANCE RESOURCE PARTNERS, L.P. - COMMON UNIT...","$15,001 - $50,000",Energy,32500.0


In [13]:
df = pd.DataFrame(transactions)

df['Disclosed'] = pd.to_datetime(df['Disclosed'])
df['Traded'] = pd.to_datetime(df['Traded'])

df.head()

,Politician Profile,Stock Ticker,Transaction,Disclosed,Traded,Description,Politician,House Information,Stock,Amount Range,Industry,Amount
0,https://www.quiverquant.com/congresstrading/po...,AAPL,Purchase,2024-07-09,2024-06-28,NaN,Josh Gottheimer,House-G000583-2712,APPLE INC. - COMMON STOCK,"$1,001 - $15,000",Information Technology,8000.0
1,https://www.quiverquant.com/congresstrading/po...,AVGO,Purchase,2024-07-09,2024-06-28,NaN,Josh Gottheimer,House-G000583-2723,BROADCOM INC. - COMMON STOCK,"$1,001 - $15,000",Information Technology,8000.0
2,https://www.quiverquant.com/congresstrading/po...,DE,Sale,2024-07-09,2024-06-28,NaN,Josh Gottheimer,House-G000583-2734,DEERE & COMPANY COMMON STOCK,"$1,001 - $15,000",Industrials,8000.0
3,https://www.quiverquant.com/congresstrading/po...,LOW,Sale,2024-07-09,2024-06-28,NaN,Josh Gottheimer,House-G000583-2764,"LOWE'S COMPANIES, INC. COMMON STOCK","$1,001 - $15,000",Consumer Discretionary,8000.0
4,https://www.quiverquant.com/congresstrading/po...,MAR,Sale,2024-07-09,2024-06-28,NaN,Josh Gottheimer,House-G000583-2768,MARRIOTT INTERNATIONAL - CLASS A COMMON STOCK,"$1,001 - $15,000",Consumer Discretionary,8000.0


In [37]:
# Handle missing values before using str.contains
df[df['Stock Ticker'].notna() & df['Stock Ticker'].str.contains('\.')].nunique()

Politician Profile    21
Stock Ticker          32
Transaction            5
Disclosed             68
Traded                78
Description            5
Politician            21
House Information     90
Stock                 27
Amount Range           5
Industry               0
Amount                 5
dtype: int64

In [36]:
# prompt: df.head() with stock ticker = (if any . in stock ticker)

df[df['Stock Ticker'].str.contains('\.')].head()


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [19]:
data = yf.download("AAPL", start="2020-01-01", end="2021-01-01")
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.960472,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.251137,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.826851,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.484337,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.650337,132079200


In [28]:
recent_data = yf.download("BRK-B", period="5d")
recent_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-22,435.049988,438.109985,432.119995,435.980011,435.980011,2995700
2024-07-23,437.339996,437.920013,433.470001,434.010010,434.010010,2393000
2024-07-24,433.989990,435.410004,430.730011,432.799988,432.799988,3143700
2024-07-25,431.600006,439.630005,431.600006,433.290009,433.290009,3334300
2024-07-26,435.660004,439.000000,434.100006,437.660004,437.660004,2716500


In [51]:
all_stock_data.to_csv('/content/drive/MyDrive/Upwork/Stock Data Analysis/all_stock_data.csv', index=False)

In [56]:
stock_data

Price        Adj Close                      Close                       High  \
Ticker               A          L   P           A          L   P           A   
Date                                                                           
2020-01-02   84.799675  52.331497 NaN   85.949997  52.959999 NaN   86.349998   
2020-01-03   83.438141  52.133873 NaN   84.570000  52.759998 NaN   85.330002   
2020-01-06   83.684799  51.461945 NaN   84.820000  52.080002 NaN   84.820000   
2020-01-07   83.941315  51.224796 NaN   85.080002  51.840000 NaN   85.260002   
2020-01-08   84.770073  51.234669 NaN   85.919998  51.849998 NaN   86.470001   
...                ...        ...  ..         ...        ...  ..         ...   
2024-07-22  133.419998  78.080002 NaN  133.419998  78.080002 NaN  133.929993   
2024-07-23  130.649994  78.349998 NaN  130.649994  78.349998 NaN  137.660004   
2024-07-24  134.160004  77.919998 NaN  134.160004  77.919998 NaN  134.429993   
2024-07-25  134.979996  78.309998 NaN  134.979996  78.309998 NaN  138.369995   
2024-07-26  137.320007  80.089996 NaN  137.320007  80.089996 NaN  140.289993   

Price                             Low                       Open             \
Ticker              L   P           A          L   P           A          L   
Date                                                                          
2020-01-02  53.000000 NaN   85.199997  52.459999 NaN   85.900002  52.720001   
2020-01-03  52.810001 NaN   84.500000  52.279999 NaN   84.669998  52.450001   
2020-01-06  52.540001 NaN   83.599998  52.020000 NaN   84.000000  52.380001   
2020-01-07  52.299999 NaN   83.940002  51.810001 NaN   83.959999  51.970001   
2020-01-08  52.580002 NaN   85.199997  51.799999 NaN   85.959999  52.049999   
...               ...  ..         ...        ...  ..         ...        ...   
2024-07-22  78.559998 NaN  131.399994  77.510002 NaN  133.059998  77.970001   
2024-07-23  78.589996 NaN  130.490005  78.089996 NaN  137.610001  78.180000   
2024-07-24  78.599998 NaN  130.179993  77.820000 NaN  130.869995  78.500000   
2024-07-25  79.690002 NaN  133.449997  77.900002 NaN  133.949997  78.260002   
2024-07-26  80.339996 NaN  136.080002  78.769997 NaN  136.690002  78.809998   

Price            Volume               
Ticker       P        A        L   P  
Date                                  
2020-01-02 NaN  1410500  1081400 NaN  
2020-01-03 NaN  1118300   829300 NaN  
2020-01-06 NaN  1993200   786600 NaN  
2020-01-07 NaN  1684700   759500 NaN  
2020-01-08 NaN  1847600   904800 NaN  
...         ..      ...      ...  ..  
2024-07-22 NaN  2009400   327100 NaN  
2024-07-23 NaN  1640800   428400 NaN  
2024-07-24 NaN  2066700   570300 NaN  
2024-07-25 NaN  1338900   457300 NaN  
2024-07-26 NaN  1717700   649500 NaN  

[1149 rows x 18 columns]

In [71]:
import yfinance as yf
import pandas as pd

# Fetch historical prices for AAPL
start_date = "2020-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')  # Today's date

stock_data = yf.download(["AAPL"], start=start_date, end=end_date, auto_adjust=False)

stock_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,74.207466,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,73.486023,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,74.071579,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,73.723213,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.909149,132079200
...,...,...,...,...,...,...
2024-07-22,227.009995,227.779999,223.089996,223.960007,223.960007,48201800
2024-07-23,224.369995,226.940002,222.679993,225.009995,225.009995,39960300
2024-07-24,224.000000,224.800003,217.130005,218.539993,218.539993,61777600


In [62]:
stock_data.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1149 entries, 2020-01-02 to 2024-07-26
Series name: Adj Close
Non-Null Count  Dtype  
--------------  -----  
1149 non-null   float64
dtypes: float64(1)
memory usage: 18.0 KB


In [80]:
all_stock_data_v2['AAPL']

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,73.192005,74.269234,72.932584,74.207466,135480400
2020-01-03,73.416844,74.264292,73.256250,73.486023,146322800
2020-01-06,72.586693,74.111111,72.329739,74.071579,118387200
2020-01-07,74.081460,74.343354,73.498379,73.723213,108872000
2020-01-08,73.419317,75.217987,73.419317,74.909149,132079200
...,...,...,...,...,...
2024-07-22,227.009995,227.779999,223.089996,223.960007,48201800
2024-07-23,224.369995,226.940002,222.679993,225.009995,39960300
2024-07-24,224.000000,224.800003,217.130005,218.539993,61777600


In [73]:
import yfinance as yf
import pandas as pd

# Assuming 'transactions' is your DataFrame with a 'Stock Ticker' column
unique_tickers = transactions['Stock Ticker'].unique()

# Fetch historical prices for all unique tickers
start_date = "2020-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')  # Today's date

all_stock_data_v3 = {}
for ticker in unique_tickers:
    try:
        stock_data = yf.download([ticker], start=start_date, end=end_date, auto_adjust=True)[["Adj Close"]]
        all_stock_data_v3[ticker] = stock_data
    except KeyError:
        print(f"Could not fetch data for {ticker}")

[*********************100%%**********************]  1 of 1 completed


Could not fetch data for AAPL


[*********************100%%**********************]  1 of 1 completed


Could not fetch data for AVGO


[*********************100%%**********************]  1 of 1 completed


...

Could not fetch data for BLK
Could not fetch data for BRK-B


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Could not fetch data for CCI


Could not fetch data for CENTA


KeyboardInterrupt: 

In [ ]:
multi_data = yf.download(["AAPL", "MSFT"], start="2020-01-01", end="2021-01-01")
multi_data

In [38]:
import yfinance as yf
from datetime import datetime





# Function to fetch stock price on or around a specific date
def get_stock_price(symbol, date):
    stock = yf.Ticker(symbol)
    try:
        # Try fetching data for the exact date, if not available, extend range
        start_date = "2021-01-01"
        end_date = datetime.strptime(date, "%Y-%m-%d")
        df = stock.history(start=start_date, end=end_date)
        #print(start_date, end_date)
        if df.empty:
            print(f"No data found for {symbol} on {date}")
            return None

        # Use the closest available trading day to the transaction date
        closest_price = df['Close'].iloc[0]
        return closest_price
    except Exception as e:
        print(f"Error fetching data for {symbol} on {date}: {e}")
        return None



In [48]:
# prompt: change stock ticker BRK.B to BRK-B in df

transactions['Stock Ticker'] = transactions['Stock Ticker'].replace('BRK.B', 'BRK-B')



# Clean and process tickers and dates
#transactions['Stock Ticker'] = transactions['Stock Ticker'].str.replace('$', '').str.strip()
transactions['Traded'] = pd.to_datetime(transactions['Traded']).dt.strftime('%Y-%m-%d')
transactions['Stock Ticker'] = transactions['Stock Ticker'].astype(str) # Convert 'Stock Ticker' column to string type


# Add columns for buy and sell prices
transactions['Transaction Price'] = transactions.apply(
    lambda row: get_stock_price(row['Stock Ticker'], row['Traded']), axis=1)

# Save the updated data with transaction prices
transactions.to_csv('transactions_with_prices.csv', index=False)
transactions.head()

ERROR:yfinance:$SGEN: possibly delisted; No timezone found


No data found for SGEN on 2023-10-27


ERROR:yfinance:$SGEN: possibly delisted; No timezone found


No data found for SGEN on 2023-06-26


ERROR:yfinance:$SGEN: possibly delisted; No timezone found


...

ERROR:yfinance:GE: Invalid input - start date cannot be after end date. startDate = 1609477200, endDate = 1517979600


No data found for GE on 2018-02-07


ERROR:yfinance:LLY: Invalid input - start date cannot be after end date. startDate = 1609477200, endDate = 1517979600


No data found for LLY on 2018-02-07


KeyboardInterrupt: 

In [47]:
print(transactions.dtypes)

Politician Profile     object
Stock Ticker           object
Transaction            object
Disclosed              object
Traded                 object
Description            object
Politician             object
House Information      object
Stock                  object
Amount Range           object
Industry               object
Amount                float64
dtype: object


In [ ]:
# Save the DataFrame to a CSV file in your Google Drive

df.to_csv('/content/drive/MyDrive/Upwork/Stock Data Analysis/transaction_scraped.csv', index=False)

In [ ]:
# Types of columns in Dataframe
df.dtypes

In [ ]:
# Length of Dataframe
len(df)

In [ ]:
df.describe(include='all')

In [ ]:
# Summary statistics for numerical columns
df.describe()

In [ ]:


from IPython.display import clear_output
import sys
import time
from urllib.parse import quote
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
import yfinance as yf
from google.colab import drive
# ### Initialization and Imports
# Update the system
!sudo apt -y update
# Install required packages
!sudo apt install -y wget curl unzip
# Download and install libu2f-udev dependency for Chrome
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
# Download and install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb

# Download and install ChromeDriver
!wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
# Install selenium and chromedriver_autoinstaller
!pip install selenium chromedriver_autoinstaller

#clear_output()
# Add chromedriver to system path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


# Configure Chrome options for headless browsing
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Install ChromeDriver automatically
chromedriver_autoinstaller.install()

# Initialize Chrome driver
driver = webdriver.Chrome(options=chrome_options)
#Scraping

# Initialize list to store trade data
trade_data = []
# ### Data Scraping
# Navigate to the QuiverQuant website
driver.get('https://www.quiverquant.com/congresstrading/')
# Parse the HTML content
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract links to politician profiles
politician_links = []

for politician_link in soup.find_all('a', attrs={'class': 'flex-row-center'}):
    politician_links.append('https://www.quiverquant.com/' + quote(politician_link.get('href').replace('../', '')))

print(f'Found {len(politician_links)} politicians')
print(f'First politician: {politician_links[0]}')
# Create a progress bar
pbar = tqdm(politician_links)

# Iterate over politician links and extract trade data
for politician_link in pbar:
    while True:
        driver.get(politician_link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        script_tag = soup.find('script', string=re.compile('let tradeData ='))

        try:
            # Extract trade data from script tag
            script_content = script_tag.string
            pbar.set_description(f'Loaded {politician_link}')
            break
        except:
            # Handle errors and retry loading the page
            pbar.set_description(f'Failed to load {politician_link}. Retrying in 10 seconds...')
            time.sleep(10)

    # Extract trade data using regular expression
    trade_data_match = re.search(r'let tradeData = (\[.*?\]);', script_content, re.DOTALL)
    trade_data_json = trade_data_match.group(1)

    # Parse trade data from JSON
    trade_data_by_politician = json.loads(trade_data_json)

    # Append trade data to the main list
    for td in trade_data_by_politician:
        trade_data.append(
            {
                'Politician Profile': politician_link,
                'Stock Ticker': td[0],
                'Transaction': td[1],
                'Disclosed': td[2],
                'Traded': td[3],
                'Description': td[4],
                'Politician': td[6],
                'House Information': td[7],
                'Stock': td[8],
                'Amount Range': td[10],
                'Industry': td[13],
                'Amount': td[14]
            }
        )

    time.sleep(1)
# ### Data Exploration
# Mount Google Drive
drive.mount('/content/drive')
# Create a Pandas DataFrame from the trade data
df = pd.DataFrame(trade_data)

# Convert date columns to datetime objects
df['Disclosed'] = pd.to_datetime(df['Disclosed'])
df['Traded'] = pd.to_datetime(df['Traded'])

# Display the first few rows of the DataFrame
df.head()
# Save the DataFrame to a CSV file in your Google Drive

df.to_csv('/content/drive/MyDrive/Upwork/Stock Data Analysis/transaction_scraped.csv', index=False)
# Types of columns in Dataframe
df.dtypes
# Length of Dataframe
len(df)
# Summary statistics for all columns
df.describe(include='all')
# Summary statistics for numerical columns
df.describe()


In [ ]:
df.sample(10)

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
# Shape of the DataFrame
df.shape

In [ ]:
# Column names
df.columns

In [ ]:
# Data types of each column
df.dtypes

In [ ]:
# Number of non-null values in each column
df.count()

In [ ]:
df.describe()

In [ ]:
# Summary statistics for all columns, including categorical ones
df.describe(include='all')

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Unique values in the 'Transaction' column
df['Transaction'].unique()

In [ ]:
# Value counts for the 'Transaction' column
df['Transaction'].value_counts()

In [ ]:
# Filter the DataFrame for purchase transactions
purchase_df = df[df['Transaction'] == 'Purchase']

In [ ]:
# Group the data by 'Politician' and count the number of transactions
transactions_by_politician = df.groupby('Politician')['Transaction'].count()

In [ ]:
# Sort the transactions by politician in descending order
transactions_by_politician_sorted = transactions_by_politician.sort_values(ascending=False)

In [ ]:
# Group the data by 'Stock Ticker' and calculate the average amount
average_amount_by_stock = df.groupby('Stock Ticker')['Amount'].mean()

In [ ]:
# Sort the average amounts by stock in descending order
average_amount_by_stock_sorted = average_amount_by_stock.sort_values(ascending=False)

In [ ]:
# Filter the DataFrame for transactions related to a specific stock
specific_stock_df = df[df['Stock Ticker'] == 'AAPL']

In [ ]:
# Calculate the time difference between disclosure and trade dates
df['Disclosure_Trade_Difference'] = (df['Disclosed'] - df['Traded']).dt.days

In [ ]:
# Distribution of disclosure-trade time differences
df['Disclosure_Trade_Difference'].describe()

In [ ]:
df['Trade Date'] = df['Traded'].dt.date

In [ ]:
# Function to scrape data
def scrape_trading_data():
    url = "https://www.quiverquant.com/congresstrading/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = []

    # Find the relevant table or data section in the HTML
    trades_table = soup.find('table', {'id': 'trading-table'})

    # Extract table rows
    for row in trades_table.find_all('tr')[1:]:
        cells = row.find_all('td')
        data.append({
            'Politician': cells[0].text.strip(),
            'Stock Symbol': cells[1].text.strip(),
            'Transaction Type': cells[2].text.strip(),
            'Transaction Date': cells[3].text.strip(),
            'Amount': cells[4].text.strip()
        })

    return pd.DataFrame(data)

In [ ]:
# prompt: # Get historical data from YahooFinance
# def get_historical_prices(symbol):
#     stock = yf.Ticker(symbol)
#     hist = stock.history(period="max")
#     return hist
# give me this one from 2021 01 01 use start date as 2021-01-01 and end date as today as getting stock.history

import pandas as pd
# Get historical data from YahooFinance
def get_historical_prices(symbol, start_date='2021-01-01', end_date=pd.Timestamp.today().strftime('%Y-%m-%d')):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    return hist


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf

# Get historical data from YahooFinance
def get_historical_prices(symbol, start_date='2021-01-01', end_date=pd.Timestamp.today().strftime('%Y-%m-%d')):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    return hist


# Function to calculate ROI
def calculate_roi(trades, historical_prices):
    roi_data = []
    for _, trade in trades.iterrows():
        symbol = trade['Stock Ticker']
        transaction_type = trade['Transaction']
        transaction_date = trade['Transaction Date']
        amount = float(trade['Amount'].replace('$', '').replace(',', ''))

        # Complete trade logic
        if transaction_type == 'Buy':
            buy_price = historical_prices[symbol].loc[transaction_date]['Close']
            sell_price = historical_prices[symbol].iloc[-1]['Close']  # Current price
            roi = (sell_price - buy_price) / buy_price * 100

        elif transaction_type == 'Sale':
            sell_price = historical_prices[symbol].loc[transaction_date]['Close']
            buy_price = historical_prices[symbol].iloc[0]['Close']  # Earliest price
            roi = (sell_price - buy_price) / buy_price * 100

        roi_data.append({
            'Politician': trade['Politician'],
            'Stock Ticker': symbol,
            'Transaction Type': transaction_type,
            'Transaction Date': transaction_date,
            'ROI': roi
        })

    return pd.DataFrame(roi_data)

# Main function to execute the workflow
def main():
    trades = df
    unique_symbols = trades['Stock Ticker'].unique()
    historical_prices = {symbol: get_historical_prices(symbol) for symbol in unique_symbols}
    roi_data = calculate_roi(trades, historical_prices)
    roi_data.to_csv('congress_trading_roi.csv', index=False)

if __name__ == "__main__":
    main()
